##TPOT Classification

In [0]:
import numpy as np
import pandas as pd

google = pd.read_csv('google_result.csv')
ibm = pd.read_csv('ibm_result.csv')
azure = pd.read_csv('azure_result.csv')

In [4]:
dict_score = {'google': (np.array(google.score,dtype=np.float64)),'ibm': np.array(ibm['score'].replace({'neutral':0}),dtype=np.float64), 'azure': np.array(azure.score),'label': list(google['label'].replace({'positive':'0','negative':'1','neutral':'2'})) }
df_scores = pd.DataFrame(dict_score)
df_scores.head()


azure  google       ibm label
0  0.887542     0.0  0.553561     0
1  0.832603     0.1  0.000000     0
2  0.500000     0.0  0.000000     2
3  0.500000     0.0 -0.533934     0
4  0.500000     0.0  0.341296     0

In [0]:
df_scores.to_csv('file.csv')

In [0]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
X = df_scores.iloc[:,:-1]
y = df_scores.iloc[:,-1]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1152, 3), (495, 3), (1152,), (495,))

In [55]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=60, population_size=40)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.5337556247396633
Generation 2 - Current best internal CV score: 0.5337556247396633
Generation 3 - Current best internal CV score: 0.5346553060451569
Generation 4 - Current best internal CV score: 0.5346553060451569
Generation 5 - Current best internal CV score: 0.5346553060451569
Generation 6 - Current best internal CV score: 0.5346553060451569
Generation 7 - Current best internal CV score: 0.5346553060451569
Generation 8 - Current best internal CV score: 0.5346553060451569
Generation 9 - Current best internal CV score: 0.5346553060451569
Generation 10 - Current best internal CV score: 0.5346553060451569
Generation 11 - Current best internal CV score: 0.5355136431103906
Generation 12 - Current best internal CV score: 0.5355136431103906
Generation 13 - Current best internal CV score: 0.5355136431103906
Generation 14 - Current best internal CV score: 0.5355136431103906
Generation 15 - Current best internal CV score: 0.5355136431103906
Gene

In [0]:
tpot.export('tpot_sentiment_pipeline.py')

##AutoSklearn Classification

In [0]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier()
cls.fit(X_train, y_train)

In [0]:
predictions = cls.predict(X_test)
from sklearn.metrics import accuracy_score
print(str(100*accuracy_score(predictions,y_test))+ " is the accuracy for AutoML SkLearn")

##H2o.ai Classification

In [0]:
!pip install h2o

In [9]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmppa3uehey
  JVM stdout: /tmp/tmppa3uehey/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmppa3uehey/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


--------------------------  ---------------------------------------------------
H2O cluster uptime:         02 secs
H2O cluster timezone:       Etc/UTC
H2O data parsing timezone:  UTC
H2O cluster version:        3.22.1.6
H2O cluster version age:    8 days
H2O cluster name:           H2O_from_python_unknownUser_s868c0
H2O cluster total nodes:    1
H2O cluster free memory:    2.938 Gb
H2O cluster total cores:    2
H2O cluster allowed cores:  2
H2O cluster status:         accepting new members, healthy
H2O connection url:         http://127.0.0.1:54321
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, Core V4
Python version:             3.6.7 final
--------------------------  ---------------------------------------------------

In [11]:
dataset = h2o.import_file("file.csv")

train,test,valid= dataset.split_frame(ratios=[.7, .15])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
aml = H2OAutoML(max_models=30, seed=99,max_runtime_secs =6000)

In [13]:
x = train.columns
y = "label"
x.remove(y)

train[y] = train[y].asfactor()
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [14]:
lb = aml.leaderboard
print (lb)

model_id mean_per_class_error logloss rmse mse 
 
 
 StackedEnsemble_BestOfFamily_AutoML_20190322_121428 0.555452 0.860269 0.551818 0.304503 
 GBM_3_AutoML_20190322_121428 0.558904 0.851839 0.549988 0.302487 
 GBM_4_AutoML_20190322_121428 0.561134 0.85648 0.549852 0.302337 
 XGBoost_grid_1_AutoML_20190322_121428_model_1 0.563609 0.86802 0.567531 0.322091 
 GBM_2_AutoML_20190322_121428 0.564092 0.849531 0.549917 0.302408 
 StackedEnsemble_AllModels_AutoML_20190322_121428 0.564627 0.869828 0.54565 0.297734 
 XGBoost_1_AutoML_20190322_121428 0.567554 0.859371 0.563348 0.317362 
 XGBoost_2_AutoML_20190322_121428 0.568776 0.882787 0.577365 0.33335 
 XGBoost_grid_1_AutoML_20190322_121428_model_6 0.569934 0.854755 0.560194 0.313818 
 XGBoost_3_AutoML_20190322_121428 0.570501 0.889486 0.580537 0.337023

In [15]:
test_y = aml.leader.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [32]:
#test_y.export_file("preds.csv") 
h2o.export_file(test_y,"preds.csv") 
h2o.export_file(test,"test_res.csv") 
preds = pd.read_csv('preds.csv') 
test = pd.read_csv('test_res.csv') 
test = test.fillna(0)
preds = preds['predict'].astype(float)
from sklearn.metrics import accuracy_score

print("H2O auto ML accuracy score is "+str(accuracy_score(preds,test['label'])))

Export File progress: |███████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%
H2O auto ML accuracy score is 0.5726495726495726


In [33]:
from sklearn.metrics import classification_report
print(classification_report(preds,test['label']))  


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.50      0.57      0.53        83
         1.0       0.00      0.00      0.00         0
         2.0       0.76      0.58      0.65       151

   micro avg       0.57      0.57      0.57       234
   macro avg       0.42      0.38      0.40       234
weighted avg       0.67      0.57      0.61       234



In [34]:
pd.crosstab(preds,test['label'])

label     0   1   2
predict            
0.0      47   8  28
2.0      47  17  87